# Predicción de Altas de Tarjetas de Crédito
### Prueba Técnica

En este notebook se desarrolla un modelo de Machine Learning para predecir si un cliente adquirirá una tarjeta de crédito (alta_tdc). Se utilizan los archivos `train_df.csv` para el entrenamiento y `Validation_df.csv` para la generación de las predicciones finales. Además, se documentan las decisiones tomadas y se reflexiona sobre posibles mejoras futuras.

## Objetivo y Entregables

### Objetivo

El objetivo de esta prueba es desarrollar un modelo capaz de predecir si un cliente adquirirá una tarjeta de crédito, utilizando el dataset proporcionado y aplicando técnicas de preprocesamiento y modelado.

### Entregables

- Notebook con el desarrollo del modelo, que incluya:
  - Desarrollo del modelo
  - Justificación de las decisiones tomadas
  - Reflexión sobre mejoras o posibles próximos pasos
- Archivo de predicciones (.csv) con las columnas `ID_cliente` y `alta_tdc`

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Lectura del CSV de entrenamiento
train_df = pd.read_csv('train_df.csv', delimiter=';', thousands='.', decimal=',')

# Visualizamos las primeras filas del dataset
train_df.head()

## Exploración y Preprocesamiento de Datos

En esta sección se separan las variables en numéricas y categóricas. Se observa que el dataset contiene una columna de identificación (`identificador_cliente`) y la variable objetivo (`alta_tdc`), por lo que estas no se utilizarán como características.

Se realiza la conversión de las columnas categóricas a tipo `str` para evitar problemas de mezcla de tipos y se procede a la preparación de pipelines específicos para cada tipo de dato.

In [ ]:
# Definir variables objetivo e identificación
target = 'alta_tdc'
id_col = 'identificador_cliente'

# Separar características (X) y variable objetivo (y)
X = train_df.drop([target, id_col], axis=1)
y = train_df[target]

# Seleccionar columnas numéricas y categóricas
num_cols = X.select_dtypes(include=['number']).columns.tolist()
cat_cols = X.select_dtypes(exclude=['number']).columns.tolist()

print("Columnas numéricas:", num_cols)
print("Columnas categóricas (antes de conversión):", cat_cols)

# Convertir columnas categóricas a string
X[cat_cols] = X[cat_cols].astype(str)

## Construcción del Pipeline de Modelado

Para estructurar el proceso de preprocesamiento y modelado se han definido dos pipelines:

- **Pipeline para variables numéricas:**
  - Imputación con la mediana (estrategia robusta ante valores atípicos).
  - Escalado con `StandardScaler` para normalizar las características.

- **Pipeline para variables categóricas:**
  - Imputación con la moda para rellenar valores faltantes.
  - Transformación a variables dummy (One-Hot Encoding) con manejo de categorías desconocidas.

Estos pipelines se combinan mediante un `ColumnTransformer`, y se integran en un pipeline final que incluye un clasificador de Bosques Aleatorios (RandomForestClassifier) configurado con 100 estimadores y una semilla fija para garantizar reproducibilidad.

In [ ]:
# Pipeline para variables numéricas
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Pipeline para variables categóricas
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combinar ambos pipelines con ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, num_cols),
    ('cat', categorical_transformer, cat_cols)
])

# Pipeline completo: preprocesamiento + modelo
clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

## Entrenamiento y Evaluación del Modelo

Se realiza la división del dataset de entrenamiento en conjuntos de entrenamiento y prueba (80-20). Posteriormente, se entrena el modelo completo y se evalúa mediante la obtención de un reporte de clasificación y la precisión global.

El reporte obtenido es el siguiente:

In [ ]:
# División en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenamiento del modelo
clf.fit(X_train, y_train)

# Predicción y evaluación
y_pred = clf.predict(X_test)

print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred))
print("Precisión:", accuracy_score(y_test, y_pred))

Se observa que el modelo tiene una alta precisión global, aunque la clase minoritaria (alta_tdc = 1) presenta un recall más bajo, lo que indica posibles mejoras en la detección de dicha clase.

## Justificación de las Decisiones Tomadas

- **Imputación y escalado para variables numéricas:** La estrategia de imputar con la mediana es robusta ante valores atípicos. El escalado permite que las variables numéricas estén en la misma escala, lo cual es importante para algoritmos basados en distancia y para la convergencia de algunos modelos.

- **Imputación y One-Hot Encoding para variables categóricas:** Utilizamos la moda para rellenar los valores faltantes en variables categóricas, y el One-Hot Encoding permite representar estas variables de forma adecuada sin asumir un orden implícito.

- **Selección del clasificador:** Se optó por un RandomForestClassifier por su capacidad para manejar conjuntos de datos heterogéneos, su robustez frente al overfitting y su interpretabilidad parcial a través de la importancia de variables.

- **División de los datos:** La separación en conjuntos de entrenamiento y prueba (80-20) permite evaluar el desempeño del modelo en datos no vistos y estimar su capacidad de generalización.

El reporte de clasificación sugiere que el modelo tiene un buen desempeño global, aunque se podría trabajar en mejorar la detección de la clase minoritaria (alta_tdc = 1).

## Reflexión y Próximos Pasos

Si bien los resultados obtenidos son prometedores, se pueden considerar las siguientes mejoras y próximos pasos:

- **Optimización de Hiperparámetros:** Realizar un ajuste fino (grid search o random search) para optimizar parámetros del RandomForest (número de estimadores, profundidad máxima, etc.).

- **Manejo del Desbalance de Clases:** Dado que la clase minoritaria tiene un recall inferior, se podría aplicar técnicas de sobremuestreo (por ejemplo, SMOTE) o submuestreo de la clase mayoritaria.

- **Validación Cruzada:** Implementar validación cruzada para evaluar de forma más robusta la capacidad de generalización del modelo.

- **Ingeniería de Variables:** Revisar el diccionario de variables para crear nuevas características o transformar las existentes que puedan aportar mayor información al modelo.

- **Modelos Alternativos:** Probar otros algoritmos (por ejemplo, XGBoost, LightGBM) que podrían ofrecer mejoras en la predicción de la clase minoritaria.

Estas acciones pueden ayudar a mejorar tanto la precisión como la capacidad de generalización del modelo, especialmente en la detección de clientes con alta probabilidad de adquirir la tarjeta de crédito.

In [ ]:
# Generación de predicciones para el conjunto de validación

# Se asume que el archivo 'Validation_df.csv' tiene una estructura similar al de entrenamiento
val_df = pd.read_csv('Validation_df.csv', delimiter=';', thousands='.', decimal=',')

# Conservamos la columna de identificación
id_val = val_df['identificador_cliente']

# Para las predicciones, se eliminan las columnas que no corresponden a las características
X_val = val_df.drop(['identificador_cliente'], axis=1)

# Generar las predicciones usando el pipeline entrenado
predicciones = clf.predict(X_val)

# Crear un DataFrame con las predicciones, renombrando la columna de ID según lo requerido
df_pred = pd.DataFrame({
    'ID_cliente': id_val,
    'alta_tdc': predicciones
})

# Guardar las predicciones en un archivo CSV
df_pred.to_csv('predicciones.csv', index=False)
print("Archivo de predicciones generado: predicciones.csv")